In [2]:
######################################################################
###############   IMPORTING THE NECESSARY PACKAGES  ##################
######################################################################

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [3]:
######################################################################
##################   DEVICE CONFIGURATION ############################
######################################################################

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
######################################################################
##################  HYPTERPARAMETER DEFINITION #######################
######################################################################

batch_size = 100
num_epochs = 80
learning_rate = 0.001

In [5]:
######################################################################
##################  PREPROCESSING OF IMAGE ###########################
######################################################################

transform = transforms.Compose([transforms.Pad(4), transforms.RandomHorizontalFlip(),
                               transforms.RandomCrop(32), transforms.ToTensor()])

In [6]:
######################################################################
#####################  PREPARING THE TRAINING DATA ###################
######################################################################

train_dataset = torchvision.datasets.CIFAR10(root = './data', train = True,
                                             transform = transform, 
                                             download = True)
test_dataset = torchvision.datasets.CIFAR10(root = './data', train = False,
                                            transform = transforms.ToTensor()
                                           )

trainLoader = torch.utils.data.DataLoader(dataset = train_dataset, 
                                                batch_size = batch_size, 
                                                shuffle = True)
testLoader = torch.utils.data.DataLoader(dataset = test_dataset,
                                               batch_size = batch_size,
                                              shuffle = False)

0it [00:00, ?it/s]

100%|█████████▉| 170057728/170498071 [00:25<00:00, 19634105.08it/s]

In [19]:
######################################################################
################# RESIDUAL BLOCK AND RESNET MODEL ####################
######################################################################

'''
The size of the input doesn't changes through this convolution
filter. Size_output = (size_input + 2*padding(1) - filter(3))/stride - 1
'''
def conv3x3(in_channels, out_channels, stride = 1):
    return(nn.Conv2d(in_channels, out_channels, kernel_size = 3, 
                    stride = stride, padding = 1, bias = False))


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride = 1, downsample = None):
        super(ResidualBlock, self).__init__()
        self.conv1 = (conv3x3(in_channels, out_channels, stride = 1))
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace = True)
        self.conv2 = (conv3x3(out_channels,out_channels, stride = 1))
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Downsample is defined in the make_layer() method of ResNet class 
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return(out)
        

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(in_channels = 3, out_channels = 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace = True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        # Downsample is basically a layer of residual blocks, if 
        # in_channels == out_channels, or stride == 1
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(conv3x3(self.in_channels, out_channels,
                                              stride=stride), nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        
        for i in range(1, blocks):
            layers.append(block(self.in_channels, out_channels, stride, downsample))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = slef.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out
    
model = ResNet(ResidualBlock, [2,2,2]).to(device)

In [21]:
######################################################################
###########  DEFINING THE LOSS CRITERION AND OPTIMIZER  ##############
######################################################################

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
######################################################################
########################  THE TRAINING LOOP  #########################
######################################################################

total_step = len(train_loader)
curr_lr = learning_rate
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainLoader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward Pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        